# Install Required Software

In [1]:
#Mambaforge
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
!bash Mambaforge-$(uname)-$(uname -m).sh -b -u -p $HOME/mambaforge

#Trimmomatic, Fastqc, bowtie, samtools, rsem
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda trimmomatic fastqc bowtie samtools rsem

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 90.4M  100 90.4M    0     0  54.1M      0  0:00:01  0:00:01 --:--:-- 82.2M
PREFIX=/home/jupyter/mambaforge
Unpacking payload ...
Extracting python-3.10.6-ha86cf86_0_cpython.tar.bz2
Extracting _libgcc_mutex-0.1-conda_forge.tar.bz2
Extracting ca-certificates-2022.9.24-ha878542_0.tar.bz2
Extracting ld_impl_linux-64-2.39-hc81fddc_0.tar.bz2
Extracting libstdcxx-ng-12.2.0-h46fd767_19.tar.bz2
Extracting pybind11-abi-4-hd8ed1ab_3.tar.bz2
Extracting tzdata-2022f-h191b570_0.tar.bz2
Extracting libgomp-12.2.0-h65d4601_19.tar.bz2
Extracting _openmp_mutex-4.5-2_gnu.tar.bz2
Extracting libgcc-ng-12.2.0-h65d4601_19.tar.bz2
Extracting bzip2-1.0.8-h7f98852_4.tar.bz2
Extracting c-ares-1.18.1

# Download Files

We need to create the folders we will use, as well as download the fastq and reference files that will be used for analysis. 

In [53]:
## Create Directories to hold files
!mkdir -p data
!mkdir -p data/raw_fastq
!mkdir -p data/trimmed
!mkdir -p data/fastqc
!mkdir -p data/reference
!mkdir -p data/rsem

## Download Fastq files

# Download run SA161911
!gsutil cp gs://hon_350/example_truncated_fastq/* data/raw_fastq

## Adapter sequences file for trimmomatic
!gsutil cp gs://hon_350/reference/TruSeq3-PE.fa data/reference/TruSeq3-PE.fa

## (Optional) Download reference files for building transcriptome index
#!gsutil cp gs://hon_350/referebce/GRCz11.dna_sm.toplevel.fa.gz data/reference/GRCz11.dna_sm.toplevel.fa.gz
#!gsutil cp gs://hon_350/referebce/GRCz11.108.gtf.gz data/reference/GRCz11.108.gtf.gz
#!echo 'unzipping...'
#!gzip -d data/reference/GRCz11.dna_sm.toplevel.fa.gz
#!gzip -d data/reference/GRCz11.107.gtf.gz
#!echo 'finished unzipping.'

## Download and unzip premade index files
!gsutil cp gs://hon_350/reference/rsem_grcz11.108_transcriptome.tar.gz data/reference/rsem_grcz11.108_transcriptome.tar.gz
!echo 'unzipping...'
!tar -xzvf data/reference/rsem_grcz11.108_transcriptome.tar.gz
!rm data/reference/rsem_grcz11.108_transcriptome.tar.gz
!echo 'finished unzipping.'

Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60184_S1_L004_R1_001.fastq.gz...
Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60184_S1_L004_R2_001.fastq.gz...
Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60185_S2_L004_R1_001.fastq.gz...
/ [3 files][  5.4 MiB/  5.4 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60185_S2_L004_R2_001.fastq.gz...
Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60186_S3_L004_R1_001.fastq.gz...
Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60186_S3_L004_R2_001.fastq.gz...
Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60187_S4_L004_R1_001.fastq.gz...
Copying gs://hon

# Trimmomatic

Trimmomatic is used to trim fastq files to remove adapters, and low quality score bases in reads.

A nice guide on trimmomatic parameters: https://datacarpentry.org/wrangling-genomics/03-trimming/index.html 

Trimmomatic works on one paired or single end run at a time.

However we have dozens of paired-end reads. 

To speed things up, this example 'pipes in' a list of run ids into trimmomatic to run sequentially.

In [104]:
# Run trimmomatic

# trimmomatic works on one paired-end reads file
# at a time. an example might look like this:

#!trimmomatic PE -threads 2 data/raw_fastq/trunc_SE6151_SA60191_S8_L004_R1_001.fastq.gz data/raw_fastq/trunc_SE6151_SA60191_S8_L004_R2_001.fastq.gz data/trimmed/SA60191_1_trimmed.fastq.gz data/trimmed/SA60191_1_orphans.fastq.gz data/trimmed/SA60191_2_trimmed.fastq.gz data/trimmed/SA60191_2_orphans.fastq.gz ILLUMINACLIP:data/reference/TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

# for the sake of this example a list is 
# created using filenames in the fastq
# directory, and those names are piped in
# iteratively to the command function

# a list of the runs is made
!ls data/raw_fastq/*R1* |  sed 's|data/raw_fastq/||g' | sed 's|1_001.fastq.gz||g' > run_ids.txt

#what the list looks like
!cat run_ids.txt
!echo ''

#the list is piped into trimmomatic
!cat run_ids.txt |  xargs -I {} trimmomatic PE -threads 2 data/raw_fastq/{}1_001.fastq.gz data/raw_fastq/{}2_001.fastq.gz data/trimmed/{}1_trimmed.fastq.gz data/trimmed/{}1_orphans.fastq.gz data/trimmed/{}2_trimmed.fastq.gz data/trimmed/{}2_orphans.fastq.gz ILLUMINACLIP:data/reference/TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

trunc_SE6151_SA60184_S1_L004_R
trunc_SE6151_SA60185_S2_L004_R
trunc_SE6151_SA60186_S3_L004_R
trunc_SE6151_SA60187_S4_L004_R
trunc_SE6151_SA60188_S5_L004_R
trunc_SE6151_SA60189_S6_L004_R
trunc_SE6151_SA60190_S7_L004_R
trunc_SE6151_SA60191_S8_L004_R
TrimmomaticPE: Started with arguments:
 -threads 2 data/raw_fastq/trunc_SE6151_SA60184_S1_L004_R1_001.fastq.gz data/raw_fastq/trunc_SE6151_SA60184_S1_L004_R2_001.fastq.gz data/trimmed/trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz data/trimmed/trunc_SE6151_SA60184_S1_L004_R1_orphans.fastq.gz data/trimmed/trunc_SE6151_SA60184_S1_L004_R2_trimmed.fastq.gz data/trimmed/trunc_SE6151_SA60184_S1_L004_R2_orphans.fastq.gz ILLUMINACLIP:data/reference/TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detecte

# Fastqc

Fastqc is a tool to inspect the quality of reads in fastq files.

In [107]:
# Run fastqc
#>add multiqc
!fastqc -o data/fastqc data/trimmed/*

# Example of in-window display of fastqc output file (SA60184).
from IPython.display import IFrame
IFrame(src='./data/fastqc/trunc_SE6151_SA60184_S1_L004_R1_trimmed_fastqc.html', width=800, height=600)


Started analysis of trunc_SE6151_SA60184_S1_L004_R1_orphans.fastq.gz
Approx 25% complete for trunc_SE6151_SA60184_S1_L004_R1_orphans.fastq.gz
Approx 50% complete for trunc_SE6151_SA60184_S1_L004_R1_orphans.fastq.gz
Approx 80% complete for trunc_SE6151_SA60184_S1_L004_R1_orphans.fastq.gz
Analysis complete for trunc_SE6151_SA60184_S1_L004_R1_orphans.fastq.gz
Started analysis of trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 5% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 10% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 15% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 20% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 25% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 30% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 35% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 40% complete for trunc_SE6151_SA60184_S1_L004_R1_trimme

# Rsem

In order to map reads to genes, and quantify read counts, an index must be created. In this case, we are using rsem for index creation and quantifying read counts.

The output and error log for this software is too large to display in the cell below, so is stored in a text log file 'rsem/rsem_log.txt' instead.

For the example truncated set this can finish within a few minutes, but with the actual data set this will take around eight hours to fully run.

In [ ]:
## Build transcriptome reference

#!rsem-prepare-reference --gtf data/reference/GRCz11.108.gtf --bowtie -p 8 data/reference/GRCz11.dna_sm.toplevel.fa data/reference/zebrafish_rsem/GRCz11_108 


In [ ]:
# to make writing commands easier,
# again we will make a list and 'pipe' the list
# into rsem.
# first make a list
!ls data/trimmed/*R1*trimmed* |  sed 's|data/trimmed/||g' | sed 's|.fastq.gz||g' > trimmed_ids.txt
# then run rsem
!echo 'running caculate counts...'
!cat trimmed_ids.txt |  xargs -I {} rsem-calculate-expression -p 16 --strandedness reverse data/trimmed/{}.fastq.gz data/reference/zebrafish_rsem/GRCz11_108 data/rsem/{} >data/rsem/rsem_log.txt 2>&1
!echo 'finished'

running caculate counts...


# Read count file outputs

Rsem outputs readcount files (XXXXX.genes.results) which are tables that contain various information, including readcounts in transcripts per kilobase million (TPM) and fragments per kilobase of exon per million mapped fragments (FPKM) format.

In [124]:
# Example of read count table output
# (sorted by top 10 expressed genes)
!head -1 $(ls data/rsem/*.genes.results | head -n1) | column -t
!sort -nrk 6 $(ls data/rsem/*.genes.results | head -n1) |  column -t | head -10  | column -t


gene_id  transcript_id(s)  length  effective_length  expected_count  TPM  FPKM
ENSDARG00000114494  ENSDART00000187218  165.00   46.17    97237.90   129867.69  172702.57
ENSDARG00000087732  ENSDART00000122203  298.00   179.17   332862.26  114565.05  152352.58
ENSDARG00000113445  ENSDART00000183292  165.00   46.17    25779.69   34430.49   45786.86
ENSDARG00000110859  ENSDART00000193467  165.00   46.17    20982.35   28023.33   37266.40
ENSDARG00000109433  ENSDART00000180315  165.00   46.17    20982.35   28023.33   37266.40
ENSDARG00000083063  ENSDART00000118247  165.00   46.17    20982.35   28023.33   37266.40
ENSDARG00000081270  ENSDART00000117461  322.00   203.17   78757.17   23904.77   31789.39
ENSDARG00000089382  ENSDART00000126982  1212.00  1093.17  400315.92  22582.71   30031.27
ENSDARG00000081580  ENSDART00000116507  72.00    1.98     705.00     21916.41   29145.21
ENSDARG00000082753  ENSDART00000115891  952.00   833.17   268968.36  19908.01   26474.37


# Upload files to bucket
In order for the files to be shared and downloaded, we need to upload them to the google bucket.

In [125]:
## Upload to bucket

!gsutil cp data/rsem/*.genes.results gs://hon_350/example_truncated_results/


Copying file://data/rsem/trunc_SE6151_SA60184_S1_L004_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
Copying file://data/rsem/trunc_SE6151_SA60185_S2_L004_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
Copying file://data/rsem/trunc_SE6151_SA60186_S3_L004_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
Copying file://data/rsem/trunc_SE6151_SA60187_S4_L004_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
- [4 files][ 10.4 MiB/ 10.4 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://data/rsem/trunc_SE6151_SA60188_S5_L004_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
Copying file://data/rsem/trunc_SE6151_SA60189_S6_L004_R1_

# scratch

In [6]:
!gsutil cp gs://kinglab-maine-zfish/WHIM/* data/example_fastq/

Copying gs://kinglab-maine-zfish/WHIM/SE6151_SA60184_S1_L004_R1_001.fastq.gz...
Copying gs://kinglab-maine-zfish/WHIM/SE6151_SA60184_S1_L004_R2_001.fastq.gz... 
Copying gs://kinglab-maine-zfish/WHIM/SE6151_SA60185_S2_L004_R1_001.fastq.gz... 
/ [3 files][  2.3 GiB/  2.3 GiB]  346.4 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://kinglab-maine-zfish/WHIM/SE6151_SA60185_S2_L004_R2_001.fastq.gz...
Copying gs://kinglab-maine-zfish/WHIM/SE6151_SA60186_S3_L004_R1_001.fastq.gz... 
Copying gs://kinglab-maine-zfish/WHIM/SE6151_SA60186_S3_L004_R2_001.fastq.gz... 
Copying gs://kinglab-maine-zfish/WHIM/SE6151_SA60187_S4_L004_R1_001.fastq.gz... 
Copying gs://kinglab-maine-zfish/WHIM/SE6151_SA60187_S4_L004_R2_001.fastq.gz... 
Copying gs://kinglab-

In [17]:
#!cd 
!gzip -d data/example_fastq/SE6151_SA60185_S2_L004_R1_001.fastq.gz

In [ ]:
!gzip -d data/example_fastq/*.fastq.gz

In [21]:
!sed -i '100001,$ d' data/example_fastq/*

^C


In [23]:
!find . -type f -name "*001.fastq" -print0 | xargs -0 sed -i '100001,$ d'

In [35]:
#import os
os.chdir('/home/jupyter/Example_Whim_Notebook')

In [29]:
!for f in *; do mv "$f" "trunc_$f"; done

In [24]:
!df

Filesystem     1K-blocks     Used Available Use% Mounted on
udev            16460384        0  16460384   0% /dev
tmpfs            3293944     8584   3285360   1% /run
/dev/sda1      102955604 10016368  88630428  11% /
tmpfs           16469708        0  16469708   0% /dev/shm
tmpfs               5120        0      5120   0% /run/lock
tmpfs           16469708        0  16469708   0% /sys/fs/cgroup
/dev/sda15        126710     6014    120696   5% /boot/efi
/dev/sdb       102626232 18035408  84574440  18% /home/jupyter


In [42]:
!gsutil cp data/example_fastq/* gs://hon_350/raw_fastq/

Copying file://data/example_fastq/trunc_SE6151_SA60184_S1_L004_R1_001.fastq [Content-Type=application/octet-stream]...
Copying file://data/example_fastq/trunc_SE6151_SA60184_S1_L004_R2_001.fastq [Content-Type=application/octet-stream]...
Copying file://data/example_fastq/trunc_SE6151_SA60185_S2_L004_R1_001.fastq [Content-Type=application/octet-stream]...
Copying file://data/example_fastq/trunc_SE6151_SA60185_S2_L004_R2_001.fastq [Content-Type=application/octet-stream]...
\ [4 files][ 34.5 MiB/ 34.5 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://data/example_fastq/trunc_SE6151_SA60186_S3_L004_R1_001.fastq [Content-Type=application/octet-stream]...
Copying file://data/example_fastq/trunc_SE6151_SA60186_S3_L004_R2_001.fa

In [50]:
!gsutil cp gs://hon_350/raw_fastq/* data/raw_fastq


Copying gs://hon_350/raw_fastq/trunc_SE6151_SA60184_S1_L004_R1_001.fastq...
Copying gs://hon_350/raw_fastq/trunc_SE6151_SA60184_S1_L004_R2_001.fastq...     
Copying gs://hon_350/raw_fastq/trunc_SE6151_SA60185_S2_L004_R1_001.fastq...     
Copying gs://hon_350/raw_fastq/trunc_SE6151_SA60185_S2_L004_R2_001.fastq...     
- [4 files][ 34.5 MiB/ 34.5 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://hon_350/raw_fastq/trunc_SE6151_SA60186_S3_L004_R1_001.fastq...
Copying gs://hon_350/raw_fastq/trunc_SE6151_SA60186_S3_L004_R2_001.fastq...     
Copying gs://hon_350/raw_fastq/trunc_SE6151_SA60187_S4_L004_R1_001.fastq...     
Copying gs://hon_350/raw_fastq/trunc_SE6151_SA60187_S4_L004_R2_001.fastq...     
Copying gs://hon_350/raw_fast

In [51]:
!gzip data/raw_fastq/*

In [52]:
!gsutil cp data/raw_fastq/* gs://hon_350/example_truncated_fastq/

Copying file://data/raw_fastq/trunc_SE6151_SA60184_S1_L004_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying file://data/raw_fastq/trunc_SE6151_SA60184_S1_L004_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying file://data/raw_fastq/trunc_SE6151_SA60185_S2_L004_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying file://data/raw_fastq/trunc_SE6151_SA60185_S2_L004_R2_001.fastq.gz [Content-Type=application/octet-stream]...
- [4 files][  7.6 MiB/  7.6 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://data/raw_fastq/trunc_SE6151_SA60186_S3_L004_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying file://data/raw_fastq/trunc_SE6151_SA60186_S3_L004_R2_001.fastq.gz [C